In [1]:


import re
import time
from collections import defaultdict
 
import pandas as pd
import requests
from lxml import etree
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
 
 

In [4]:
# 使用chrome开发者模式
option = webdriver.ChromeOptions()
# 屏蔽webdriver特征
option.add_experimental_option(
    "excludeSwitches", ["enable-automation"]
)  
# 禁用启用Blink运行时的功能，关闭网页页面的反扒校验
option.add_argument("--disable-blink-features=AutomationControlled")

option.add_argument("--no-sandbox") # 表示禁用沙盒模式
# 沙盒模式：每一个标签页都是一个沙盒（sandbox），以“防止恶意软体自行执行安装”或“利用一个分页影响其他的分页”
# 关闭web沙盒的命令，有可能导致浏览恶意网站时，被入侵。 
option.add_argument("--disable-dev-shm-usage") # 启动浏览器时，添加参数--disable-dev-shm-usage。该参数使用本地local/tmp代替/dev/shm作为 Chrome 的运行空间，local/tmp比/dev/shm有更大的空间，可以使Cypress运行时，不容易因为一个文件的测试用例数多，导致内存溢出的问题。

# 指定chromedriver的路径
chrome_service = Service(r'C:/Users/89859/AppData/Local/Google/Chrome/Application/chromedriver.exe')
# 创建Chrome WebDriver对象
driver = webdriver.Chrome(service=chrome_service, options=option)
# Selenium执行cdp命令，没有出现验证框就能登录说明已经成功屏蔽网页对selenium的识别。

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
                Object.defineProperty(navigator, 'webdriver', {
                    get: () => undefined
                })
                """
})

driver.set_page_load_timeout(25) # 用于设置页面加载超时,指定时间内未加载出页面则会报错
driver.delete_all_cookies()  # 删除所有 cookie 信息
url = (
    "https://www.dianping.com/" 
    # "https://www.dianping.com/shop/laVusV94nCHPy3Eg/review_all" 
)
driver.get(url)
time.sleep(1)    # 使用chrome开发者模式


In [5]:

# 打开目标网页
driver.get("https://www.dianping.com/shop/H1xjHbW7Wnp24NrB/review_all/p101")

# 定义等待时间
wait_time = 5

# 定义滚动次数
scroll_count = 100  

# 定义要遍历的 div 范围
start_index = 1
end_index = 15

print("-----------------waiting page ready-----------------")


# 定义点击展开评价函数
def click_expand_buttons():
    try:
        more_buttons = driver.find_elements(By.CLASS_NAME, "more-words")
        for button in more_buttons:
            driver.execute_script("arguments[0].click();", button)
        print("All '展开评价' buttons clicked")
    except NoSuchElementException:
        print("No '展开评价' buttons found")


# 初始化评论列表
all_contents = []

# 开始循环抓取评论
# for _ in range(scroll_count):
for _ in range(20):
    # 等待页面加载完成
    time.sleep(wait_time)

    # 点击展开评价
    click_expand_buttons()

    print("-----------------start collecting comments-----------------")

    # 遍历 div 范围
    contents = []
    for i in range(start_index, end_index + 1):
        try:
            d = driver.find_elements(
                By.XPATH,
                f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]',
            )
            for element in d:
                content_members = []
                name = element.find_element_by_xpath(
                    f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]/div/div[1]/a').text
                print(name)
                # score = element.find_element_by_xpath(
                #     f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]/div/div[2]/span[2]').text
                # print(score)
                taste = element.find_element_by_xpath(
                    f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]/div/div[2]/span[2]/span[1]').text
                print(taste)
                environment = element.find_element_by_xpath(
                    f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]/div/div[2]/span[2]/span[2]').text
                print(environment)
                service = element.find_element_by_xpath(
                    f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]/div/div[2]/span[2]/span[3]').text
                print(service)
             
             
                # 尝试在多个位置找到评论内容
                for j in range(3, 6):  # 假设评论的位置在 div[3] 到 div[5] 之间
                    try:
                        comments = element.find_element_by_xpath(
                            f'/html/body/div/div[2]/div[2]/div[3]/div[3]/div[3]/ul/li[{i}]/div/div[{j}]').text
                        if comments:
                            break  # 找到非空评论后跳出循环
                    except NoSuchElementException:
                        continue  # 如果当前位置没有评论，则继续尝试下一个位置
                
                print(comments)
                content_members.append(name)
                # content_members.append(score)
                content_members.append(taste)
                content_members.append(environment)
                content_members.append(service)
                content_members.append(comments)
                contents.append(content_members)
        except NoSuchElementException:
            print("没有找到评论")

    print("current page contents", contents)

    # 将当前页面的评论添加到所有评论列表中
    all_contents.extend(contents)
    print("all_contents", all_contents)
    # 转换为DataFrame
    df = pd.DataFrame(all_contents, columns=['name','taste','environment','service', 'comment'])

    # 将DataFrame写入Excel文件
    df.to_excel('D:/Python/SCGC/大众点评dianping/raw_content_dianping_barbecue_test200.xlsx', index=False)

    # 构造下一页的 URL
    next_page_url = driver.find_element(By.CLASS_NAME, "NextPage").get_attribute("href")

    # 打开下一页的 URL
    if next_page_url:
        driver.get(next_page_url)
    else:
        print("没有找到下一页按钮")
        break

# # 打印所有抓取到的评论
# print("all contents", all_contents)

-----------------waiting page ready-----------------
All '展开评价' buttons clicked
-----------------start collecting comments-----------------
喵了个咪
没有找到评论
Ricardo
没有找到评论
林曦熙
口味：3.5


C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  name = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:62: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  taste = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:65: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  environment = element.find_element_by_xpath(


环境：3.5
服务：3.5
在大梅沙走完海滨栈道 刚好到饭点 之前想吃这家一直要等位 抱着试试的心态去了 不需要等位就打算尝试一下 「烤调味牛肋条」 点了三个肉类都是未解冻的 牛肋条的口感还行 「人参鸡汤」 参鸡汤 说实话很一般 跟在韩国吃的比起来 味道完全不同 「烤原味猪颈肉」 猪颈肉很肥 最后都没怎么吃
收起评价
慢大王


C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:68: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  service = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:76: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comments = element.find_element_by_xpath(


口味：4.5
环境：4.5
服务：5.0
[薄荷]环境： 刚开业的期间人拍的有点多，毕竟是沙头角很少出现的烤肉，之前想吃烤肉还得出沙头角才能吃得到，也是终于有烤肉开进了沙头角 味道还不错，小菜的话给的很多，吃烤肉前恐怕都会吃饱哈哈 五花肉是值得推荐的！剪完之后每一条烤熟一点吃起来很好吃！满足！
收起评价
点小评3804958159
口味：2.0
环境：3.0
服务：1.0
这里菜也不咋的 第一次去这里吃东西 特别说的是服务 叫了无数遍服务员都不看一下 还有的就是直接走去另外一桌去
我爱孙颖莎
没有找到评论
阿环呀
口味：2.5
环境：2.5
服务：4.0
两个女生点了两个肉一个芝士年糕，不要点太多了，芝士多的吃到发腻
匿名用户
口味：4.5
环境：4.0
服务：4.5
正好在盐田办事，附近搜到这家韩式烤肉评价还不错就来了。这家店生意很旺啊，下午一点多人流不减，排了一会队才进来，有根据排队时长赠送不同食物，这点很好哦！ 「烤调味牛肋条」「烤卷五花肉」「石锅拌饭」味道很不错 「炒年糕」比较甜腻了，我不太喜欢 小菜是无限供应的，海草很不错。 走的时候还吃了一根店里的雪糕，服务也很值得表扬!
收起评价
壹仟萬
口味：1.0
环境：1.0
服务：1.0
超级无敌差的一家烤肉店，点菜很久才上菜就算了，还把肉烤焦了，鸡蛋和芝士本来是和肉一起吃的，我肉吃完了，鸡蛋芝士都不给我上来。青菜没有洗，里面都粘着脏东西，太可怕了。小菜的出品完全不地道韩国菜，全部偏味精味，偏甜，吃一口小菜要喝一杯水解渴的节奏！严重人手不足，很多服务员都说不清楚，刚来的，应该是请的兼职吧～等了一个半小时，就这出品和服务，太辣🐔了～撇开性价比，服务和出品是不会再来了，瑟瑟发抖…… [环境]油烟味很大，一顿饭下来除了不好吃，服务巨慢，还要欠缺舒适感，本年度吃的最揪心的一顿饭～
收起评价
富
没有找到评论
匿名用户
没有找到评论
FCHICKEN
口味：4.5
环境：4.0
服务：4.5
以前在中山吃的烤肉，终于也在深圳吃到啦！八号线开通，可以去看个海吃个烤肉哟～不过记得五点提前拿号，跟姜虎东一样人多的 小菜都很足，吃不够可以无限续，吃小菜吃到饱哦。肉🥩的话跟其他烤肉没啥区别～ 点了个海鲜豆腐汤，有点不对劲，汤端上来看起来就像用重新火锅底料做的一样，太油了，跟外面吃的不太一样，这个希望改进下哦
收起评价
floath

...

出差在外，随便找的。 吃完觉得点评的评价有点低，其实还能吃的呀。 烤肉本身质量尚可，可能是不对深圳人民口味？ 不过五花切的稍微有点薄，吃的不是很爽。 两侧的蛋烤熟了嫩嫩的，也是我的大爱之一。 总结是，感觉应该在8分多一点，可以尝试的。
收起评价
cicizhuang
口味：4.5
环境：4.0
服务：4.5
盐田新开的韩国烤肉店 在深圳有几家分店，不过好像在东莞的最出名。 调味牛排🥩价格有点高，口感还可以。 牛舌分量很大，不过不要烤太熟，会硬。 五花肉也还行 泡菜饼有点油 比较喜欢石锅拌饭 这家店的烤炉和烟囱感觉有点旧的样子 蘸的酱感觉不太正宗 小菜泡菜也还可以 作为沙头角第一家韩国烤肉店挺好的🏻 值得多次打卡
收起评价
kk
口味：5.0
环境：5.0
服务：5.0
*在东莞跟深圳其他地方吃过gogiya 刚好盐田区开了就来试试 今天到店里 没有其他评论说的那么差诶 服务态度好好！ *🍖 「烤卷五花肉」五花肉是每次的必点！烤嫩一点&烤焦一点都很好吃 后者没那么腻 蘸芝士吃跟醋吃都好吃！ 「烤调味牛肋条」不要烤太熟 好嫩！ 「辛拉面」这个一般诶 不太推荐 「小菜」可以续 推荐土豆泥跟花生 花生甜甜的好好吃 *🧃 个人觉得桃子味跟葡萄味的好喝！谢谢老板送的饮料
收起评价
TY_
口味：5.0
环境：5.0
服务：5.0
味道很不错哦，服务也很好，一直有帮忙烤肉！小菜，泡菜饼，肉，还有韩国饮料的味道都不错！我很少给五星，这家真的不错，买单时会认真问顾客的意见。我说感觉五花肉有点肥，老板就送了两瓶饮料谢谢啦！会再来吃的～
收起评价
匿名用户
口味：3.0
环境：2.5
服务：4.5
①这家店我排队是第一次排了一个多小时都不会动的一家店，十分让我生气得是，我在门口看了好久，部分顾客是在里面坐着玩手机等朋友的，菜没上锅没热，这样对我们这些在外面排队的是不公平的，要是他朋友一小时半小时不到，我们岂不是也要因为他朋友没到，在门口等待这么长时间，我觉得是不太公平的，希望店家合理安排排队的问题，门口排队是真的热。 ②后面这个免费得配菜及服务员帮我们烤肉，我是比较满意的 ③石锅饭很好吃，饮料也很不错，烤肉也还好，五花肉有点肥，老其他的环境各方面都是可以的 下次还会考虑，希望老板重视上面的缺点
收起评价
babyluang805
没有找到评论
赵公子
口味：2.5
环境：3.0
服务：3

C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  name = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:62: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  taste = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:65: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  environment = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:68: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  service = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:76: DeprecationWarning: find_elem

服务：5.0
打卡，第一次吃烤肉好吃，开心，就是排队的人太多了
heyjj_913
没有找到评论
雅子小公主
没有找到评论
点小评0899142497
口味：1.0
环境：1.0
服务：1.0
过号等三桌就可以在吃，这种规则，我等4小时都等不到。。有啥用？
点小评3774844245
口味：3.0
环境：3.0
服务：1.0
不按顺序排队，过号了也照样优先
洗澡去_3520
口味：4.5
环境：4.5
服务：5.0
第二次来，新店活动，服务员会贴心得帮你减掉烤焦的肉的边缘。推荐烤大肠
山茶
口味：5.0
环境：5.0
服务：5.0
泡菜饼好吃😋 小菜也很好吃 😋感觉小菜都可以把人吃饱哈哈哈哈 石锅拌饭也好吃 炒年糕一般 服务也很好
CASTLE宝宝
口味：5.0
环境：5.0
服务：5.0
菜品整体味道不错，服务比较热情，配菜免费续，点了雪花牛肉，辣猪五花，调味牛肋条和牛舌，都挺好吃的，烤起来比较嫩～
jiaping88
没有找到评论
porker123
口味：2.5
环境：3.0
服务：3.0
如果，其实，真的，没必要。 小菜不正宗不好吃 烤肉烤得老不咋地 服务真的让人无语 环境真的也很闷热 就，没必要拍这个队，费这个心机，花这个钱。
yuyuyuiefh
口味：3.5
环境：3.5
服务：3.5
盐田终于开了一家韩国烤肉，恰好抢到万客50抵100的券，忘记在在网上取号，6点到现场拿的号，等了2个半钟终于到我了……现场油烟稍微有点大，坐在位置上就会有人送几碟小菜过来，还不错，「精品雪花龙牛肉」这个真的好吃，肉很嫩「烤卷五花肉」烤的有点nong，觉得一般 打完折下来100出头，两个人吃，🉑️
收起评价
匿名用户
口味：3.5
环境：3.5
服务：3.5
今天下班跟办公室同事临时决定要来吃烤肉，然后六点预约排号，等了两个小时才吃上，真的是太热闹啦。店内环境感觉有点拥挤，很嘈杂，不过服务还不错，会帮忙烤肉，基本上就负责吃就行啦，东西还行吧，小菜很多，培根好吃，金针菇好吃，其他的比较无感。其实感觉没吃什么就很饱了哈哈哈
收起评价
最爱大姐姐小弟弟
没有找到评论
CC小姐姐
口味：5.0
环境：5.0
服务：5.0
打卡🇰🇷韩国烤肉，这家店铺在我印象中是刚开没有多久，人流也很多，常常经过的时候都看到有在排队的现象。 [薄荷]环境： 感觉很普通，透明的玻璃可以从外面看到里面。

C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  name = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:62: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  taste = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:65: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  environment = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:68: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  service = element.find_element_by_xpath(
C:\Users\89859\AppData\Local\Temp\ipykernel_31164\1540883787.py:76: DeprecationWarning: find_elem

在壹海城二楼新开的一家烤肉店「GOGIYA韩国传统烤肉(盐田壹海城店)」环境超好，烤肉超好次。时不时会有员工会来帮你烤肉，服务超好，两个人的战斗力实在是太小了还没吃到什么就饱了，分量太足了 看到人家推荐的雪花龙牛肉，真心不错。太赞了 豆腐汤就一般般，下次要去尝试一下海带汤 沙律，配菜太棒了，还能无限续盘
收起评价
JIAN
口味：4.0
环境：4.0
服务：5.0
终于去打卡啦！壹海城新开的烤肉店最近还有7.8折真的很不错，服务员很热情！店长都亲自来给我们烤肉而且小菜无限加，泡菜是偏酸口的，太合我口味啦芝士，蛋羹免费无限加！去过很多烤肉店芝士都要加钱的！这个烤肉店老板太良心啦 推荐菜品： ✨烤猪大肠 真的是我吃过最棒的了！太好吃了配上孜然粉！忘了说这里还可以自助调料真的很不错 ✨泡菜炒饭 一个大煎蛋配上满满的泡菜和猪肉的炒饭 太香了！ 下次要来试试冷面和炸酱面！还有烤牛舌
收起评价
kumiyoyo
口味：4.5
环境：4.5
服务：5.0
吃完了才记得拍照不好意思了。 服务：服务我们的“试用期” 的小卓很棒，轻声细语 烤肉来说牛雪花最好吃 每桌送了两瓶橙汁和菠萝汁汽水 炒年糕也真的好吃 送了很多小菜：泡菜，沙拉、萝卜、豆芽、凉皮，还可以续噢 全场折扣7.8折～ 三大一小折后348元 建议周末的话还是中午来 ，11:10 就来拿号吧，否则像我昨晚6点拿的号（前面39桌）就吃不到的结局了
收起评价
饕餮
口味：5.0
环境：5.0
服务：5.0
非常好，就是人很多，芝士和蛋液随便加，小菜无限续，是和朋友聚餐，五花肉炒新鲜，雪花牛也好嫩，good
Kaiser
口味：3.0
环境：4.0
服务：4.5
位置：在one mall二楼，肯德基隔壁，非常好找，建议大家早一点去，需要排队 [薄荷]环境： 韩国烤肉的标准装修，还有韩国的店员 「烤卷五花肉」五花肉我个人感觉一般般，烤出来不是很香 「烤调味牛肋条」牛肋条的调味十分不错，但是牛肉没什么牛肉味 「冷面」冷面还不错，凉凉的，在夏天吃十分爽！ 「海鲜豆腐脑辣汤」这个还不错
收起评价
雨中冷百合
口味：5.0
环境：5.0
服务：4.5
还真是不错，试业的时候去的，等位能等到脚抽筋的那种
Ivan
口味：5.0
环境：5.0
服务：5.0
很开心沙头角终于有韩国烤肉 各方面都十分正宗
熊仔chiwah
口味：4.0
环境：4.